# Table Of Content

# Project Overview


# Data Collection and Initial Processing



## Dataset Overview
The dataset used in this project originates from a Portuguese retail bank and contains detailed records of telemarketing campaigns conducted between 2008 and 2013. These campaigns were aimed at promoting long-term deposit subscriptions among existing and potential customers.
The data was collected and organized across two versions:
bank.zip – containing data from the initial campaigns between 2008 and 2010.
bank-additional.zip – an extended version collected between 2008 and 2013 with richer socio-economic indicators and additional campaign details.
Together, the datasets include up to 150 attributes, encompassing a broad range of customer demographics, banking product information, campaign interaction details, and external macroeconomic variables.
The dataset is widely recognized for benchmarking predictive modeling techniques in marketing analytics and serves as an excellent real-world example for classification problems in data science.
The target variable, y, indicates whether the telemarketing call resulted in a successful sale of a term deposit (yes) or not (no).

## Data Description
The dataset’s structure integrates multiple domains of information that collectively influence telemarketing outcomes. The features can be grouped into the following main categories:
1. Customer Demographics
These variables describe the socio-demographic profile of each client:
age – Client’s age (numeric).
job – Type of occupation (e.g., admin, technician, blue-collar, services, etc.).
marital – Marital status (married, single, divorced).
education – Education level (basic, secondary, tertiary, unknown).
default – Indicates if the client has credit in default (yes, no).
housing – Has a housing loan (yes, no).
loan – Has a personal loan (yes, no).
2. Campaign and Communication Attributes
These describe the telemarketing contact details and campaign context:
contact – Communication type (cellular or telephone).
month – Last contact month of the year.
day_of_week – Last contact day of the week.
duration – Duration of the last call in seconds.
campaign – Number of contacts performed during this campaign for the client.
pdays – Number of days since the client was last contacted in a previous campaign (-1 if never contacted).
previous – Number of contacts performed before this campaign.
poutcome – Outcome of the previous marketing campaign (e.g., success, failure, nonexistent).
3. Banking Product Details
Information about the client’s relationship with the bank and existing products:
balance – Average yearly balance in euros.
deposit subscription (y) – The target variable indicating campaign success (yes for successful subscription, no otherwise).
4. Socio-Economic Context
These external indicators reflect macroeconomic conditions at the time of each campaign:
emp.var.rate – Employment variation rate (quarterly indicator).
cons.price.idx – Consumer price index.
cons.conf.idx – Consumer confidence index.
euribor3m – Euribor 3-month rate.
nr.employed – Number of employees in the economy.

## Analytical Relevance
This dataset provides a rich foundation for:
Exploratory Data Analysis (EDA) to uncover patterns in client behavior.
Feature engineering to enhance predictive modeling.
Machine learning classification to predict y (success of the campaign).
Model interpretation using tools such as SHAP and LIME to derive actionable insights for marketing optimization.

## Project Alignment
By combining these data attributes with modern data science techniques—such as logistic regression, random forests, gradient boosting (XGBoost), and neural networks—the project aims to:
Predict telemarketing call success more accurately.
Identify key drivers of positive campaign outcomes.
Provide strategic recommendations to improve campaign efficiency and reduce operational costs.

## Data Ingestion and Integration

Following the project alignment phase, the next step focuses on assembling a clean and comprehensive dataset for analysis. Multiple CSV files containing campaign details, customer demographics, and call outcomes are ingested and merged into a single unified dataframe using Python’s pandas library. During this process, shared identifiers are used to align records across files, while NumPy and pandas utilities support validation checks to ensure consistency, resolve missing or mismatched entries, and confirm structural integrity. The resulting dataset provides a reliable foundation for the subsequent preprocessing, modeling, and analysis stages.

### Library Imports
To load the data, we need to import the pandas library which will be used to read and combine the CSV files.

Other libraries used at latter parts of project will also be imported here, to ensure a consistent and organized workflow.

In [1]:
# library imports

import pandas as pd

### Creating DataFrames from Data Sources

With pandas imported, the next step is to read in the data from the various data files.
Each source file is loaded into a separate pandas DataFrame.

This approach allows for easy inspection of each dataset before merging them into one big dataframe for a unified analysis.

In [2]:
# reading datasets into dataframes

df1 = pd.read_csv("data/bank-additional-full.csv", sep=";")
df2 = pd.read_csv("data/bank-additional.csv", sep=";")
df3 = pd.read_csv("data/bank-full.csv", sep=";")
df4 = pd.read_csv("data/bank.csv", sep=";")

### Data Inspection and Validation

After loading the datasets into separate DataFrames, the next step is to inspect and validate each one before merging.
This helps ensure that all files were read correctly and that their structures are consistent.

We begin by:

- Viewing sample records with head() to confirm data integrity.

- Checking dataset dimensions using shape.

- Reviewing data types and non-null counts with info().

- Identifying missing values using isnull().sum().

- Verifying that key identifiers (e.g., customer_id, campaign_id) are present and properly formatted.

These checks help detect potential issues early and ensure a smooth integration process in the next step.

#### df1

In [3]:
# view sample records to confirm data integrity

df1.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
7,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
8,24,technician,single,professional.course,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
# Checking dataset dimensions

df1.shape

(41188, 21)

In [5]:
# Reviewing data types and non-null counts
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [6]:
# Identifying missing values

df1.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

##### Observation
- Columns: `'age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'`
- No key/unique identifier (e.g campaign IDs) found.
- Some columns seem to have same value for all rows. Further exploration needed.

#### df2

In [7]:
# view sample records to confirm data integrity

df2.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
5,32,services,single,university.degree,no,no,no,cellular,sep,thu,...,3,999,2,failure,-1.1,94.199,-37.5,0.884,4963.6,no
6,32,admin.,single,university.degree,no,yes,no,cellular,sep,mon,...,4,999,0,nonexistent,-1.1,94.199,-37.5,0.879,4963.6,no
7,41,entrepreneur,married,university.degree,unknown,yes,no,cellular,nov,mon,...,2,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
8,31,services,divorced,professional.course,no,no,no,cellular,nov,tue,...,1,999,1,failure,-0.1,93.200,-42.0,4.153,5195.8,no
9,35,blue-collar,married,basic.9y,unknown,no,no,telephone,may,thu,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no


In [8]:
# Checking dataset dimensions

df2.shape

(4119, 21)

In [9]:
# Reviewing data types and non-null counts
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4119 entries, 0 to 4118
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4119 non-null   int64  
 1   job             4119 non-null   object 
 2   marital         4119 non-null   object 
 3   education       4119 non-null   object 
 4   default         4119 non-null   object 
 5   housing         4119 non-null   object 
 6   loan            4119 non-null   object 
 7   contact         4119 non-null   object 
 8   month           4119 non-null   object 
 9   day_of_week     4119 non-null   object 
 10  duration        4119 non-null   int64  
 11  campaign        4119 non-null   int64  
 12  pdays           4119 non-null   int64  
 13  previous        4119 non-null   int64  
 14  poutcome        4119 non-null   object 
 15  emp.var.rate    4119 non-null   float64
 16  cons.price.idx  4119 non-null   float64
 17  cons.conf.idx   4119 non-null   f

In [10]:
# Identifying missing values

df2.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

##### Observation
- Columns: `'age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'`
- No key/unique identifier (e.g campaign IDs) found.
- Most of the columns and corresponding categorical classes are present in df1 as well

#### df3

In [11]:
# view sample records to confirm data integrity

df3.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [12]:
# Checking dataset dimensions

df3.shape

(45211, 17)

In [13]:
# Reviewing data types and non-null counts
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [14]:
# Identifying missing values

df3.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

##### Observation
- Columns: `'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y'`
- No key/unique identifier (e.g campaign IDs) found.
- less columns than df1 and df2
- Similar columns with df1 and df2
- Some columns seem to have same value for all rows. Further exploration needed.

#### df4

In [15]:
# view sample records to confirm data integrity

df4.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
5,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no
6,36,self-employed,married,tertiary,no,307,yes,no,cellular,14,may,341,1,330,2,other,no
7,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,-1,0,unknown,no
8,41,entrepreneur,married,tertiary,no,221,yes,no,unknown,14,may,57,2,-1,0,unknown,no
9,43,services,married,primary,no,-88,yes,yes,cellular,17,apr,313,1,147,2,failure,no


In [16]:
# Checking dataset dimensions

df4.shape

(4521, 17)

In [17]:
# Reviewing data types and non-null counts
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [18]:
# Identifying missing values

df4.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

##### Observation
- Columns: `'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y'`
- No key/unique identifier (e.g campaign IDs) found.
- less columns than df1 and df2
- Similar columns with df1 and df2

#### Tabular Summary

The shape and structure of all the dataframes are summarized in tabular form below.

In [19]:

# dictionary of all dataframes
dataframes = {
    "df1": df1,
    "df2": df2,
    "df3": df3,
    "df4": df4
}

# Function to summarize key info
def summarize_df(df):
    return {
        "Rows": df.shape[0],
        "Columns": df.shape[1],
        "Missing Values": df.isnull().sum().sum(),
        "Duplicate Rows": df.duplicated().sum(),
        "Numeric Columns": df.select_dtypes(include='number').shape[1],
        "Categorical Columns": df.select_dtypes(exclude='number').shape[1]
    }

# Generate summary table
summary = pd.DataFrame({name: summarize_df(df) for name, df in dataframes.items()}).T
summary


,Rows,Columns,Missing Values,Duplicate Rows,Numeric Columns,Categorical Columns
df1,41188,21,0,12,10,11
df2,4119,21,0,0,10,11
df3,45211,17,0,0,7,10
df4,4521,17,0,0,7,10


#### Assessing Common and Unique Columns

After inspecting each dataframe, it’s useful to identify columns which are shared and which are unique to each source.
This helps determine how the DataFrames can be aligned or merged effectively.

In [20]:
# Identify common columns
common_columns = set.intersection(*(set(df.columns) for df in dataframes.values()))
print("Common columns across all DataFrames:\n", common_columns)

Common columns across all DataFrames:
 {'duration', 'loan', 'pdays', 'y', 'marital', 'education', 'previous', 'housing', 'age', 'contact', 'month', 'job', 'default', 'campaign', 'poutcome'}


In [21]:
# Identify unique columns for each DataFrame 
print("\nUnique columns in each DataFrame:\n")
for name, df in dataframes.items():
    unique_cols = set(df.columns) - common_columns
    print(f"{name}: {unique_cols if unique_cols else 'None'}")


Unique columns in each DataFrame:

df1: {'cons.price.idx', 'nr.employed', 'cons.conf.idx', 'day_of_week', 'emp.var.rate', 'euribor3m'}
df2: {'cons.price.idx', 'nr.employed', 'cons.conf.idx', 'day_of_week', 'emp.var.rate', 'euribor3m'}
df3: {'balance', 'day'}
df4: {'balance', 'day'}


#### Key Observations

- No truly identical columns across all datasets (only similar or partially overlapping ones).

- A few important columns appear in two out of four datasets.

#### Next Steps

The outcomes of the Data Inspection and Validation across the four datasets suggest the following next steps:

- Standardization of Column Names
- Merge Datasets Vertically
- Inspect Missing Data
- Remove Columns with Excessive Missing or Non-Informative Values
- Remove Rows with Too Many Missing Values

#### Standardize Column Names

To ensure proper alignment during concatenation, column names are standardized by converting them to lowercase and replacing spaces with underscores.


In [22]:

for name, df in dataframes.items():
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')



#### Merge Datasets Vertically

The individual DataFrames are concatenated using pd.concat().
This stacks all records into a single dataset, automatically inserting NaN values where columns don’t overlap.


In [23]:

combined_df = pd.concat(dataframes.values(), ignore_index=True, sort=False)
print("Combined shape:", combined_df.shape)


Combined shape: (95039, 23)



#### Inspect Missing Data

Before removing any values, we first review the extent of missingness across all columns.


In [24]:

missing_report = (
    combined_df.isnull().sum()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'index': 'column', 0: 'missing_count'})
)
missing_report['missing_percent'] = round(missing_report['missing_count'] / len(combined_df) * 100, 2)
missing_report.head(20)



,column,missing_count,missing_percent
0,cons.price.idx,49732,52.33
1,emp.var.rate,49732,52.33
2,euribor3m,49732,52.33
3,nr.employed,49732,52.33
4,day_of_week,49732,52.33
5,cons.conf.idx,49732,52.33
6,balance,45307,47.67
7,day,45307,47.67
8,loan,0,0.00
9,default,0,0.00



This provides a quick overview of which columns contain the most missing values.


#### Remove Columns with Excessive Missing or Non-Informative Values

Columns with a large proportion of missing entries or those containing only a single constant value are dropped.


In [25]:

# Drop columns with more than 70% missing data
threshold = 0.3  
combined_df = combined_df.loc[:, combined_df.isnull().mean() < threshold]


In [26]:

# Drop columns with no variability
for col in combined_df.columns:
    if combined_df[col].nunique() <= 1:
        combined_df.drop(columns=col, inplace=True)



#### Remove Rows with Too Many Missing Values

Rows missing more than 40% of their values are removed to maintain data quality.


In [27]:

combined_df.dropna(thresh=int(0.6 * combined_df.shape[1]), inplace=True)


#### Review 
Now let's see the outcome of the cleaning steps

In [28]:

print("Shape after cleaning:", combined_df.shape)
combined_df.info()

Shape after cleaning: (95039, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95039 entries, 0 to 95038
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        95039 non-null  int64 
 1   job        95039 non-null  object
 2   marital    95039 non-null  object
 3   education  95039 non-null  object
 4   default    95039 non-null  object
 5   housing    95039 non-null  object
 6   loan       95039 non-null  object
 7   contact    95039 non-null  object
 8   month      95039 non-null  object
 9   duration   95039 non-null  int64 
 10  campaign   95039 non-null  int64 
 11  pdays      95039 non-null  int64 
 12  previous   95039 non-null  int64 
 13  poutcome   95039 non-null  object
 14  y          95039 non-null  object
dtypes: int64(5), object(10)
memory usage: 10.9+ MB


### Feature Engineering

# Exploratory Data Analysis (EDA)

# Predictive Modeling

# Prescriptive Analytics and Recommendations

# Model Deployment